# Deploy NVIDIA NIM from AWS Marketplace

NVIDIA NIM, a component of NVIDIA AI Enterprise, enhances your applications with the power of state-of-the-art large language models (LLMs), providing unmatched natural language processing and understanding capabilities. Whether you're developing chatbots, content analyzers, or any application that needs to understand and generate human language, NVIDIA NIM for LLMs has you covered.

In this example we show how to deploy the Cosmos-Reason1-7B model from AWS Marketplace

NVIDIA Cosmos Reason – an open, customizable, 7B-parameter reasoning vision language model (VLM) for physical AI and robotics - enables robots and vision AI agents to reason like humans, using prior knowledge, physics understanding and common sense to understand and act in the real world. This model understands space, time, and fundamental physics, and can serve as a planning model to reason what steps an embodied agent might take next.

Given a video and a text prompt, the model first converts the video into tokens using a vision encoder and a special translator called a projector. These video tokens are combined with the text prompt and fed into the core model, which uses a mix of LLM modules and techniques. This enables the model to think step-by-step and provide detailed, logical responses.

Please check out the [Cosmos-Reason1-7B model card](https://build.nvidia.com/nvidia/cosmos-reason1-7b/modelcard) and [NIM LLM docs](https://docs.nvidia.com/nim/large-language-models/latest/introduction.html) for more information.

## ⚠️ Disclaimer

Reasoning models often require longer inference times, which may exceed the default 60-second timeout limit for [**AWS SageMaker's non-streaming endpoints**](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_runtime_InvokeEndpoint.html). This notebook shows examples for both the streaming and non-streaming endpoints.

To avoid inference failures due to timeout:
- It is **recommended** to use the **SageMaker streaming endpoint** for this model as shown in the [Streaming inference](#Streaming-inference) section of this notebook.
- If your use case **requires** using a **non-streaming endpoint** as shown in the [Run Inference](#Run-Inference) section of this notebook, you must first contact **AWS Support** to request an increased timeout limit for your **AWS Account and Region** to avoid unexpected errors.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to one of the models listed above.


## Subscribe to the model package
To subscribe to the model package:
1. Open the model package listing page
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
import boto3, json, sagemaker, time, os
from sagemaker import get_execution_role, ModelPackage
from botocore.config import Config

config = Config(read_timeout=3600)
sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime", config=config)
region = sess.region_name

In [ ]:
# replace the arn below with the model package arn you want to deploy
nim_package = ""

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{nim_package}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{nim_package}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{nim_package}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{nim_package}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{nim_package}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{nim_package}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{nim_package}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{nim_package}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{nim_package}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{nim_package}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{nim_package}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{nim_package}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{nim_package}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{nim_package}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{nim_package}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{nim_package}",
}

region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]
model_package_arn

## Create the SageMaker Endpoint

We first define SageMaker model using the specified ModelPackageArn.

In [ ]:
# Define the model details
sm_model_name = "cosmos-reason1-7b"

# Create the SageMaker model
create_model_response = sm.create_model(
    ModelName=sm_model_name,
    PrimaryContainer={
        'ModelPackageName': model_package_arn
    },
    ExecutionRoleArn=role,
    EnableNetworkIsolation=True
)
print("Model Arn: " + create_model_response["ModelArn"])

Next we create endpoint configuration specifying instance type

In [ ]:
# Create the endpoint configuration
endpoint_config_name = sm_model_name

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': sm_model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g5.12xlarge', 
            'InferenceAmiVersion': 'al2-ami-sagemaker-inference-gpu-2',
            'RoutingConfig': {'RoutingStrategy': 'LEAST_OUTSTANDING_REQUESTS'},
            'ModelDataDownloadTimeoutInSeconds': 3600, # Specify the model download timeout in seconds.
            'ContainerStartupHealthCheckTimeoutInSeconds': 3600, # Specify the health checkup timeout in seconds
        }
    ]
)
print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to InService once the deployment is successful.

In [ ]:
# Create the endpoint
endpoint_name = endpoint_config_name
create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Run Inference

Once we have the model deployed we can use a sample text to do an inference request. For inference request format, currently NIM on SageMaker supports the OpenAI API inference protocol. For explanation of supported parameters please see [this link](https://docs.api.nvidia.com/nim/reference/nvidia-llama-3_3-nemotron-super-49b-v1-infer).

#### Non Reasoning Mode

Since Cosmos-Reason1-7B is a VLM (Vision-Language Model), we can run inference with images or videos, and get text response from the model. Additionally, Cosmos-Reason1-7B supports text-only queries. Please refer to docs [text-only-queries](https://docs.nvidia.com/nim/vision-language-models/1.4.1/examples/cosmos-reason1/api.html#text-only-queries).

**Inference with image**

In [ ]:
import base64
import mimetypes
from pathlib import Path

def encode_file_to_base64(file_path):
    """Encode a local media file to base64 for airgapped environment"""
    file_path = Path(file_path)
    
    # Guess the MIME type based on the file extension
    mime_type, _ = mimetypes.guess_type(file_path)
    if mime_type is None:
        raise ValueError(f"Could not determine MIME type for file: {file_path}")
    
    with open(file_path, "rb") as f:
        encoded_string = base64.b64encode(f.read()).decode("utf-8")
        return f"data:{mime_type};base64,{encoded_string}"

In [ ]:
image_filename = 'cat.jpg'  # Change to your image file
image_data_url = encode_file_to_base64(image_filename)

payload_model = "nvidia/cosmos-reason1-7b"
messages = [
    {
      "role":"user",
      "content":[
        {
          "type": "text",
          "text": "What is in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_data_url
          }
        }
      ]
    }
  ]

payload = {
  "model": payload_model,
  "messages": messages,
  "temperature": 0.6,
  "max_tokens": 200
}

response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

output = json.loads(response["Body"].read().decode("utf8"))
print(json.dumps(output, indent=2))

**Inference with video**

To control how frames are sampled from video inputs, sampling parameters are exposed using the top-level media_io_kwargs API field.

Either fps or num_frames can be specified (but not both at the same time). More information please refer to [sampling-and-preprocessing-parameters](https://docs.nvidia.com/nim/vision-language-models/latest/examples/cosmos-reason1/api.html#sampling-and-preprocessing-parameters)

In [ ]:
video_filename = '960_540_25fps.mp4'  # Change to your video file
video_data_url = encode_file_to_base64(video_filename)

payload_model = "nvidia/cosmos-reason1-7b"
messages = [
    {
      "role":"user",
      "content":[
        {
          "type": "text",
          "text": "What is in this video?"
        },
        {
          "type": "video_url",
          "video_url": {
            "url": video_data_url
          }
        }
      ]
    }
  ]

payload = {
  "model": payload_model,
  "messages": messages,
  "temperature": 0.6,
  "max_tokens": 200,
  "media_io_kwargs": {"video": {"fps": 1}},
}

response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

output = json.loads(response["Body"].read().decode("utf8"))
print(json.dumps(output, indent=2))

#### Reasoning mode

Following the [guidance](https://huggingface.co/nvidia/Cosmos-Reason1-7B#input) from model authors, append the following to the system prompt to encourage a long chain-of-thought reasoning response.

In [ ]:
Answer the question in the following format: <think>\nyour reasoning\n</think>\n\n<answer>\nyour answer\n</answer>.

In [ ]:
payload_model = "nvidia/cosmos-reason1-7b"
messages = [
    {   
      "role": "system",
      "content": "Answer the question in the following format: <think>\nyour reasoning\n</think>\n\n<answer>\nyour answer\n</answer>."
    },
    {
      "role":"user",
      "content":[
        {
          "type": "text",
          "text": "What is in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_data_url
          }
        }
      ]
    }
  ]

payload = {
  "model": payload_model,
  "messages": messages,
  "temperature": 0.6,
  "max_tokens": 200
}

response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

output = json.loads(response["Body"].read().decode("utf8"))
print(json.dumps(output, indent=2))

### Streaming inference

NIM on SageMaker also supports streaming inference and you can enable that by setting **`"stream"` as `True`** in the payload and by using [`invoke_endpoint_with_response_stream`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime/client/invoke_endpoint_with_response_stream.html) method.

#### Non Reasoning Mode

In [ ]:
image_filename = 'cat.jpg'  # Change to your image file
image_data_url = encode_file_to_base64(image_filename)

payload_model = "nvidia/cosmos-reason1-7b"
messages = [
    {
      "role":"user",
      "content":[
        {
          "type": "text",
          "text": "What is in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_data_url
          }
        }
      ]
    }
  ]

payload = {
  "model": payload_model,
  "messages": messages,
  "temperature": 0.6,
  "max_tokens": 200,
  "stream": True
}

response = client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
    Accept="application/jsonlines",
)

We have some postprocessing code for the streaming output.

In [ ]:
event_stream = response['Body']
accumulated_bytes = b''
stream_finished = False # flag to track termination

for event in event_stream:
    if stream_finished:
        break 

    try:
        payload_part = event.get('PayloadPart', {})
        if 'Bytes' in payload_part:
            accumulated_bytes += payload_part['Bytes']
        
        decoded_data = accumulated_bytes.decode('utf-8-sig', errors='ignore')
    
        parts = decoded_data.rpartition('\n')
        lines_to_process = parts[0]
        accumulated_bytes = parts[2].encode('utf-8', errors='ignore')
        
        for line in lines_to_process.split('\n'):
            line = line.strip()
            
            if not line:
                continue
            
            if line == "data: [DONE]":
                print("\n") 
                stream_finished = True 
                break
            
            if line.startswith('data:'):
                json_str = line[len('data:'):].strip()
                
                if json_str:
                    try:
                        data = json.loads(json_str)
                        
                        content = data.get('choices', [{}])[0].get('delta', {}).get('content', "")
                        
                        # Print only the generated content token
                        if content:
                            print(content, end='', flush=True)
                            
                    except json.JSONDecodeError:
                        continue
        
    except Exception as e:
        print(f"\n[ERROR: CRITICAL] -> Error processing stream: {e}", flush=True)
        break

print("\n[STREAM END: LOOP COMPLETED]", flush=True)

#### Reasoning mode

In [ ]:
payload_model = "nvidia/cosmos-reason1-7b"
messages = [
    {   
      "role": "system",
      "content": "Answer the question in the following format: <think>\nyour reasoning\n</think>\n\n<answer>\nyour answer\n</answer>."
    },
    {
      "role":"user",
      "content":[
        {
          "type": "text",
          "text": "What is in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_data_url
          }
        }
      ]
    }
  ]

payload = {
  "model": payload_model,
  "messages": messages,
  "temperature": 0.6,
  "max_tokens": 2000,
  "stream": True
}

response = client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
    Accept="application/jsonlines",
)

We have some postprocessing code for the streaming output.

In [ ]:
event_stream = response['Body']
accumulated_bytes = b''
stream_finished = False # flag to track termination

for event in event_stream:
    if stream_finished:
        break 

    try:
        payload_part = event.get('PayloadPart', {})
        if 'Bytes' in payload_part:
            accumulated_bytes += payload_part['Bytes']
        
        decoded_data = accumulated_bytes.decode('utf-8-sig', errors='ignore')
    
        parts = decoded_data.rpartition('\n')
        lines_to_process = parts[0]
        accumulated_bytes = parts[2].encode('utf-8', errors='ignore')
        
        for line in lines_to_process.split('\n'):
            line = line.strip()
            
            if not line:
                continue
            
            if line == "data: [DONE]":
                print("\n") 
                stream_finished = True 
                break
            
            if line.startswith('data:'):
                json_str = line[len('data:'):].strip()
                
                if json_str:
                    try:
                        data = json.loads(json_str)
                        
                        content = data.get('choices', [{}])[0].get('delta', {}).get('content', "")
                        
                        # Print only the generated content token
                        if content:
                            print(content, end='', flush=True)
                            
                    except json.JSONDecodeError:
                        continue
        
    except Exception as e:
        print(f"\n[ERROR: CRITICAL] -> Error processing stream: {e}", flush=True)
        break

print("\n[STREAM END: LOOP COMPLETED]", flush=True)

### Terminate endpoint and clean up artifacts

In [ ]:
sm.delete_model(ModelName=sm_model_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=endpoint_name)